<a href="https://colab.research.google.com/github/JakeANam/AIstudy24/blob/master/%EA%B5%90%EC%B0%A8%EA%B2%80%EC%A6%9D_%EA%B7%B8%EB%A6%AC%EB%93%9C%EC%84%9C%EC%B9%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

지금까지 배운 algorism - 훈련세트와 테스트세트를 나누어 평가를 진행
결국 test set로 평가하면 거기에 맞는 model이 만들어지는 결론이 나온다.

지금까지 train_set, test_set 만 생각했는데, 사실 검증하기 위한 set도 있어야한다

test set의 일반화 성능을 올바르게 예측하려면 가능한한 test_set를 사용하지 말아야한다

model을 만들고 나서 마지막에 딱 한번 사용하는 것이 좋다

결정tree에서 max_depte 매개변수를 사용한 hyperparameter 튜닝을 해보자

In [1]:
# 검증세트
# 테스트 세트를 사용하지 않으면 그 model이 과대적합인지 과소적합인지 어떻게 알까?
# 테스트 세트를 사용하지 않고 측정하는 간단한 방법? - 훈련세트를 또 나누는 것

# 훈련 set 60%, 검증 set와 test set 각 20% - 실무에서 많이 쓰는 기법

# test하고 싶은 매개변수를 바꿔가면서 가장 좋은 model이 나올 수 있다
# 이 매개변수를 사용해 훈련set와 검증set를 합쳐 전체 훈련 data에서 model을 다시 훈련하고
# 마지막 test에서 최종점수 평가 개시
# 훈련 set에서 model 훈련, 검증세트에서 model을 평가
# 마지막set에서 testset이용해서 최종점수 평가


In [9]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()

from sklearn.model_selection import train_test_split
# 원본 data에서 20%는 test set으로 생성
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)
# 검증 set로 20% 생성
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)

print(train_input.shape, test_input.shape)
print(sub_input.shape, val_input.shape)

(5197, 3) (1300, 3)
(4157, 3) (1040, 3)


In [14]:
# sub set과 val set를 사용해서 model 생성, 훈련 후 평가
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input,sub_target)

DecisionTreeClassifier(random_state=42)

In [15]:
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))
#과대적합 발생 - 매개변수를 바꿔서 더 좋은 model을 찾아야한다

0.9971133028626413
0.864423076923077


In [48]:
# Cross Validate: 그 만큼 검증을 많이 하므로 평가 신뢰도 증가 - 실무에서 많이 쓰게 될 것
# 검증 세트를 떼서 평가하는 과정을 여러번 반복하는 것
# 3-fold Cross Validate - 3번 반복(앞에 5, 혹은 10도 있다)
from sklearn.model_selection import cross_validate
import numpy as np

scores = cross_validate(dt, train_input, train_target, cv=5)
# cv=5 <- fold 수 변경(기본 5, 생략시 5로 설정, 2이상부터 가능)
# 이 method는 seed값 설정이 따로 없기 때문에 항상 답이 바뀔 수도 있다
print(scores)
# fit_time(훈련시간), score_time(검증시간), test_score(최종점수) 키를 가진 딕셔너리 반환

# 5개의 검증 점수의 평균을 내서 최종점수를 얻는다
print("평균점수: %f" %np.mean(scores['test_score'])) # <- 검증fold의 점수

{'fit_time': array([0.01528168, 0.00978947, 0.01003051, 0.00894928, 0.00871515]), 'score_time': array([0.00164676, 0.00177026, 0.00165558, 0.00159121, 0.00147438]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}
평균점수: 0.855300


In [56]:
# 주의! : cross_validate()는 훈련 set를 섞어서 fold를 나누지 않고 splitter 사용
# 앞서 우리는 train_test_split()함수로 전체 data를 섞은 후 훈련set 준비했다
# 만약 교차검증을 할 때 훈련 set를 섞으려면 분할기(splitter)를 지정해야 한다

# sklearn의 분할기는 교차검증에서 fold를 어떻게 나눌지 결정하고
# cross_validate()는 기본적으로 회귀모델인 KFold 분할기를 사용한다
# 분류model일 경우 target class를 골고루 나누기 위해서 stratifiedKFold 사용
from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(scores)
print("평균점수: %f" %np.mean(scores['test_score']))

{'fit_time': array([0.00982523, 0.01148057, 0.00911045, 0.0088594 , 0.00819707]), 'score_time': array([0.00142336, 0.00150943, 0.00146365, 0.00138187, 0.00118947]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}
평균점수: 0.855300


In [57]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42) # 10 fold교차 검증 수행
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(scores)
print("평균점수: %f" %np.mean(scores['test_score']))

{'fit_time': array([0.01094198, 0.00961971, 0.00986075, 0.00944304, 0.00919938,
       0.00934458, 0.00943112, 0.01060748, 0.00926304, 0.0091536 ]), 'score_time': array([0.00127625, 0.00122905, 0.00125313, 0.00124407, 0.00117922,
       0.00122094, 0.00126362, 0.00115728, 0.00119352, 0.00115418]), 'test_score': array([0.83461538, 0.87884615, 0.85384615, 0.85384615, 0.84615385,
       0.87307692, 0.85961538, 0.85549133, 0.85163776, 0.86705202])}
평균점수: 0.857418
